In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [102]:
df = pd.read_csv(links['d_learning_notes_url'])
df

,Process,Categorization,Word,Definition
0,ML Definitions,Model Type,Types of ML,"Within the machine learning paradigm, models a..."
1,ML Model,Process,Definition,ML Models is a process designed to store recor...
2,ML Project,Process Step,Goals,"It is imperative to have clear goals, specific..."
3,ML Project,Process Step,Goals,Goals of a ML Project Differ from Goals of BI ...
4,ML Project,Process Step,Problem Definition,"Define, Quantify and Articulate Problem."
...,...,...,...,...
136,Langchain,Process Step,TBD,Build a chain using the pipe operator `|` to ...
137,Langchain,Process Step,TBD,Invoke the chain with specific inputs to gener...
138,Tooling,TBD,TBD,Streamlit and Gradio are both Python framework...
139,Tooling,TBD,TBD,LangChain and LlamaIndex are complementary but...


In [86]:
from data_d_dicts import links

list_ = ['Process','Categorization','Word','Definition']

# Download Data
definition_df = pd.read_csv(links['google_definition_csv'])
notes_df = pd.read_csv(links['google_notes_csv'])

definition_df = definition_df[list_].copy()
notes_df = notes_df[list_].copy()

# Step 1. Create a list of unique Processes from Notes.
process_list = notes_df['Process'].unique().tolist()
process_list.extend([x for x in definition_df['Process'].unique() if (x not in process_list)&(pd.notna(x))])
process_map = {x:count+0 for count,x in enumerate(process_list)}

# Step 2 Create a Unique Classification List
categorization_list = [
    'Definition','Guiding Principle','Consideration','Process Step','Procedure','Expected Outcomes','Parameter','Algorithm']
